In [1]:
from deep_orderbook.shaper import BookShaper, ArrayShaper
import numpy as np
from pylab import rcParams
from tqdm.auto import tqdm

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn_image as isns
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [3]:
MARKETS = ["ETH-BTC", "BTC-USD", "ETH-USD"]

In [4]:
# itertools for asyncio !
import asyncio
from aioitertools import iter, next, map, zip
from aioitertools import enumerate

In [ ]:
from deep_orderbook.feeds.coinbase_feed import CoinbaseFeed
from deep_orderbook.replayer import ParquetReplayer

async def iter_sec(date_regexp='2024-08-06', max_samples=0):
    replayer = ParquetReplayer('data', date_regexp=date_regexp)
    async with CoinbaseFeed(
        markets=MARKETS,
        replayer=replayer,
    ) as feed:
        async for onesec in feed.one_second_iterator():
            yield onesec
            if max_samples == 0:
                break
            else:
                max_samples -= 1

async for onesec in iter_sec(max_samples=3):
    print(onesec)

In [ ]:
async def iter_shapes(date_regexp='2024-08-06', max_samples=0):
    symbol_shapers = {pair: ArrayShaper(zoom_frac=0.002) for pair in MARKETS}
    async for onesec in iter_sec(max_samples=max_samples):
        yield {
            market: await symbol_shapers[market].make_arr3d(
                onesec.symbols[market]
            )
            for market in MARKETS
        }

async for shaped in iter_shapes(max_samples=1):
    print(shaped)


In [ ]:
async def iter_gen_arrays(date_regexp='2024-08-06', max_samples=0):
    async for shapes in iter_shapes(max_samples=max_samples):
        yield shapes['ETH-USD'].transpose(1, 0, 2)
        # yield np.hstack([shapes[market] for market in MARKETS]).transpose(1, 0, 2)


async for arr in iter_gen_arrays(max_samples=1000):
    im = arr.copy()
    im[:,:,0] *= -0.2
    im[:,:,1:3] *= 1e6
    isns.imshow(im, origin='lower', cmap='coolwarm', vmin=-1, vmax=1, gray=True)
    clear_output(wait=True)
    plt.show()


In [ ]:
async def iter_shapes_t2l(date_regexp='2024-08-06', max_samples=0):
    symbol_shaper = ArrayShaper(zoom_frac=0.002)
    async for onesec in iter_sec(max_samples=max_samples):
        yield [
            await symbol_shaper.make_arr3d(onesec.symbols['ETH-USD']),
            await symbol_shaper.build_time_level_trade(),
        ]


async for shaped, t2l in iter_shapes_t2l(max_samples=1000):
    im = t2l[:,:,0]
    # isns.imshow(im.transpose(1, 0, 2), origin='lower', cmap='coolwarm', vmin=-1, vmax=1, gray=True)
    toshow = np.clip(im.T, -1000, 1000)
    
    plt.subplot(2, 1, 1)
    im = shaped.transpose(1, 0, 2)
    im[:,:,0] *= -0.2
    im[:,:,1:3] *= 1e6
    plt.imshow(im, origin='lower', cmap='coolwarm')

    plt.subplot(2, 1, 2)
    plt.imshow(toshow, cmap='nipy_spectral', origin="lower")
    plt.grid( which='both', axis='both')
    clear_output(wait=True)

    plt.show()
